In [21]:
import re
from numpy import nan
###REWORKER FOR INITIAL FILES
def fix_csv(csv_name):
    csv = open(csv_name, 'r', encoding="UTF-8")
    newcsv = open(csv_name[:-4] + "_R.csv", 'w', encoding="UTF-8")
    names = csv.readline().strip()
    newcsv.write(names.replace(' ', ',') + "\n")
    for line in csv.readlines():
        check = re.search(r'"(.*)"', line)
        if check:
            for match in check.groups():
                line = re.sub(match, match.replace(' ', '_'), line)
        splited_list = line.split(' ')[1:]
        if '' in splited_list:
           splited_list[splited_list.index('')] = str(nan)
        for i in range(len(splited_list)):
            splited_list[i] = splited_list[i].replace('_', ' ')
        newcsv.write(','.join(splited_list))
    csv.close()
    newcsv.close()

In [22]:
fix_csv('cinema_sessions.csv')
fix_csv('titanic_with_labels.csv')  

In [23]:
from os.path import exists
import numpy as np
import pandas as pd

def selectAgeGroup(age, ageGroups):
    for group in ageGroups:
        if age < group[0]:
            return group[1]
    return None

def selectDrinkType(drink, drinkGroups):
    for group in drinkGroups:
        if drink in group[0]:
            return group[1]
    return None

def selectSessionType(time, timeGroups):
    return selectAgeGroup(time, timeGroups) # :)

SEX_MALE_DEFF = ["M", "m", "М", "м"]
SEX_FEM_DEFF = ["F", "ж", "Ж", "f"]
DRINK_NORMALIZED_VALUE = 20
AGEGROUPS = [
    (18, "Child"),
    (50, "Adult"),
    (999, "Elderly")] #Возрастные группы задаются в виде (Максимальный возраст (Не включительно), Наименование группы)
DRINKSGROUPS = [
    (['Beerbeer', 'Bugbeer', 'Strong beer'], 1),
    (['Cola', 'Fanta', 'Water'], 0)] #Класификации напитков в виде (Список напитков, Значение для замены)
SESSIONSTYPES = [
    ("12:00:00.000", "Morning"),
    ("17:00:00.000", "Afternoon"),
    ("24:00:00.000", "Evening")]

if not exists('cinema_sessions_R.csv'):
    fix_csv('cinema_sessions.csv')
    fix_csv('titanic_with_labels.csv')    
sess = pd.read_csv('cinema_sessions_R.csv', encoding="UTF-8")
tit = pd.read_csv('titanic_with_labels_R.csv', encoding="UTF-8")

#Fix genders
tit.sex = tit.sex.replace(SEX_FEM_DEFF, 1).replace(SEX_MALE_DEFF, 0)
tit = tit[tit.sex.apply(str).str.isdigit()]

#Fix row_numbers
tit.row_number = tit.row_number.fillna(tit.row_number.max())

#Fix liters
litersDrunkNormalized = tit.liters_drunk[(tit.liters_drunk >= 0) & (tit.liters_drunk <= DRINK_NORMALIZED_VALUE)]
tit.loc[(tit.liters_drunk <= 0) | (tit.liters_drunk >= DRINK_NORMALIZED_VALUE), 'liters_drunk'] = int(litersDrunkNormalized.mean())

#Age's
tit.age = tit.age.apply(selectAgeGroup, ageGroups=AGEGROUPS)

#Alcohol content of beverages
tit.drink = tit.drink.apply(selectDrinkType, drinkGroups=DRINKSGROUPS)

#Add session type
tit = pd.merge(tit, sess, on="check_number").rename(columns={"session_start" : "session_type"})
tit.session_type = tit.session_type.apply(selectSessionType, timeGroups=SESSIONSTYPES)

tit

,sex,row_number,liters_drunk,age,drink,check_number,label,session_type
0,1,79.0,2,Elderly,0,3116.0,0.0,Evening
1,0,23.0,2,Child,0,8118.0,0.0,Afternoon
2,0,92.0,2,Adult,0,2264.0,1.0,Evening
3,1,52.0,5,Elderly,1,7640.0,1.0,Evening
4,1,57.0,2,Adult,1,8764.0,0.0,Afternoon
...,...,...,...,...,...,...,...,...
6662,1,100.0,3,Child,0,1022.0,1.0,Afternoon
6663,0,15.0,3,Elderly,1,6510.0,0.0,Evening
6664,1,11.0,3,Elderly,1,9510.0,1.0,Evening
6665,0,96.0,5,Elderly,1,9554.0,0.0,Afternoon
